In [62]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm
import json

# Stichwortsuche Robert Enke

In [114]:
seed_page="https://fazarchiv.faz.net/fazSearch/index/searchForm?q=robert+enke&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&maxHits=10&sorting=&toggleFilter=&dosearch=y&offset="
json_list=[]
attributes=["title","thema_1","date","text","link","autor","art"]
json_file={k:np.nan for k in attributes}

for offset in range(0,830,10):

    print("Page: {}".format(1+(offset/10)))
    scd_address_part=str(offset)+"#hitlist"
    page=requests.get(seed_page+scd_address_part)
    print(seed_page+scd_address_part)
    page_content=BeautifulSoup(page.content,"html.parser")

    for page_element in page_content.find_all("form",{"id":"multiDocSelectForm"}):

        for box_element in page_element.find_all("div",class_=re.compile("module13 clearfix*")):
            json_file={k:np.nan for k in attributes}
            #titel
            json_file["title"]=box_element.find_all("a")[0].text
            #print("title: {}".format(json_file["title"]))
            #thema_1
            json_file["thema_1"]=box_element.find_all("h2")[0].text
            #Autor
            if len(box_element.find_all("h2"))>1:
                json_file["autor"]=box_element.find_all("h2")[1].text.replace("Von ","")
            else: 
                json_file["autor"]="N/A"
            #Art
            json_file["art"]=box_element.find_all("a",class_="globalLinkButton")[0].text
            #Link
            json_file["link"]="https://fazarchiv.faz.net"+str(box_element.find_all("a",class_="globalLinkButton")[0].get("href"))
            #Kurztext
            json_file["text"]=box_element.find_all("p")[0].text[24:]
            #Datum
            json_file["date"]=box_element.find_all("li")[0].text[:-1]
            #Ressort
            json_file["ressort"]=box_element.find_all("li")[1].text
            #Länge
            try:
                json_file["laenge"]=box_element.find_all("li")[2].text[1:]
            except:
                print("extraction laenge failed")
            json_list.append(json_file)

ePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&maxHits=10&sorting=&toggleFilter=&dosearch=y&offset=0#hitlist
Page: 2.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=robert+enke&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&maxHits=10&sorting=&toggleFilter=&dosearch=y&offset=10#hitlist
Page: 3.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=robert+enke&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&maxHits=10&sorting=&toggleFilter=&dosearch=y&offset=20#hitlist
Page: 4.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=robert+enke&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&maxHits=10&sorting=&toggleFilter=&dosearch=y&offset=30#hitlist
Page: 5.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=robert+enke&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&D

In [116]:
df=pd.DataFrame(json_list)

In [122]:
def get_month(x):
    month=["Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"]
    month_dict={k:v for k,v in zip(month,range(1,13,1))}

    for month in month_dict.keys():
        if month in x:
            return month_dict[month]
            break

In [131]:
df["month"]=df["date"].apply(lambda x: get_month(x))
df["year"]=df["date"].apply(lambda x: x[-4:])
df["date"]=df.apply(lambda x: pd.to_datetime("01."+str(x["month"])+"."+str(x["year"]),format="%d.%m.%Y"),axis=1)
df.drop(["month","year"],axis=1,inplace=True)

In [132]:
df.head()

,title,thema_1,date,text,link,autor,art,ressort,laenge
0,Das neue Schönheitsideal,,2020-05-01,Eine unfreiwillig radikale Verjüngungskur verw...,https://fazarchiv.faz.net#,N/A,Zeitungsseite,Sport,1688 Wörter
1,Das neue globale Schönheitsideal des Fußballs,Deutschland bei der WM 2010,2020-05-01,"Die WM 2010 war ein Glück, das niemand ahnen k...",https://fazarchiv.faz.net/document/FAZN__20200...,"Christian Eichler, München",Artikel,FAZ.NET,1495 Wörter
2,"""Was bleibt, sind stille 1:1-Situationen""",DENK ICH AN SPORT,2020-04-01,Der Dramatiker und Schriftsteller Moritz Rinke...,https://fazarchiv.faz.net#,N/A,Zeitungsseite,Sonntagszeitung,2142 Wörter
3,"""Was bleibt, sind stille 1:1-Situationen""",Denk ich an Sport (32),2020-04-01,"Moritz Rinke, 52 Jahre, zählt zu den erfolgrei...",https://fazarchiv.faz.net/document/FAZN__20200...,Moritz Rinke,Artikel,FAZ.NET,2101 Wörter
4,Der beste Minister,,2020-03-01,Von Matthias Alexander / / Stabil wie kein Zw...,https://fazarchiv.faz.net#,N/A,Zeitungsseite,Rhein-Main-Zeitung,360 Wörter


In [134]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\FAZ\FAZ_Archiv_Query_[Robert_Enke].xlsx') as writer:
    df.to_excel(writer)

# Stichwortsuche Depression

In [135]:
seed_page="https://fazarchiv.faz.net/fazSearch/index/searchForm?q=depression&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset="
json_list=[]
attributes=["title","thema_1","date","text","link","autor","art"]
json_file={k:np.nan for k in attributes}

for offset in range(0,8270,10):

    print("Page: {}".format(1+(offset/10)))
    scd_address_part=str(offset)+"#hitlist"
    page=requests.get(seed_page+scd_address_part)
    print(seed_page+scd_address_part)
    page_content=BeautifulSoup(page.content,"html.parser")

    for page_element in page_content.find_all("form",{"id":"multiDocSelectForm"}):

        for box_element in page_element.find_all("div",class_=re.compile("module13 clearfix*")):
            json_file={k:np.nan for k in attributes}
            #titel
            json_file["title"]=box_element.find_all("a")[0].text
            #print("title: {}".format(json_file["title"]))
            #thema_1
            json_file["thema_1"]=box_element.find_all("h2")[0].text
            #Autor
            if len(box_element.find_all("h2"))>1:
                json_file["autor"]=box_element.find_all("h2")[1].text.replace("Von ","")
            else: 
                json_file["autor"]="N/A"
            #Art
            json_file["art"]=box_element.find_all("a",class_="globalLinkButton")[0].text
            #Link
            json_file["link"]="https://fazarchiv.faz.net"+str(box_element.find_all("a",class_="globalLinkButton")[0].get("href"))
            #Kurztext
            json_file["text"]=box_element.find_all("p")[0].text[24:]
            #Datum
            json_file["date"]=box_element.find_all("li")[0].text[:-1]
            #Ressort
            json_file["ressort"]=box_element.find_all("li")[1].text
            #Länge
            try:
                json_file["laenge"]=box_element.find_all("li")[2].text[1:]
            except:
                print("extraction laenge failed")
            json_list.append(json_file)

timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=7520#hitlist
Page: 754.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=depression&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=7530#hitlist
Page: 755.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=depression&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=7540#hitlist
Page: 756.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=depression&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=7550#hitlist
Page: 757.0
https://fazarchiv.faz.net/fazSearch/

In [137]:
df=pd.DataFrame(json_list)

In [138]:
df["month"]=df["date"].apply(lambda x: get_month(x))
df["year"]=df["date"].apply(lambda x: x[-4:])
df["date"]=df.apply(lambda x: pd.to_datetime("01."+str(x["month"])+"."+str(x["year"]),format="%d.%m.%Y"),axis=1)
df.drop(["month","year"],axis=1,inplace=True)

In [142]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\FAZ\FAZ_Archiv_Query_[Depression].xlsx') as writer:
    df.to_excel(writer)

# Stichwortsuche Suizid

In [143]:
seed_page="https://fazarchiv.faz.net/fazSearch/index/searchForm?q=suizid&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset="
json_list=[]
attributes=["title","thema_1","date","text","link","autor","art"]
json_file={k:np.nan for k in attributes}

for offset in range(0,3040,10):

    print("Page: {}".format(1+(offset/10)))
    scd_address_part=str(offset)+"#hitlist"
    page=requests.get(seed_page+scd_address_part)
    print(seed_page+scd_address_part)
    page_content=BeautifulSoup(page.content,"html.parser")

    for page_element in page_content.find_all("form",{"id":"multiDocSelectForm"}):

        for box_element in page_element.find_all("div",class_=re.compile("module13 clearfix*")):
            json_file={k:np.nan for k in attributes}
            #titel
            json_file["title"]=box_element.find_all("a")[0].text
            #print("title: {}".format(json_file["title"]))
            #thema_1
            json_file["thema_1"]=box_element.find_all("h2")[0].text
            #Autor
            if len(box_element.find_all("h2"))>1:
                json_file["autor"]=box_element.find_all("h2")[1].text.replace("Von ","")
            else: 
                json_file["autor"]="N/A"
            #Art
            json_file["art"]=box_element.find_all("a",class_="globalLinkButton")[0].text
            #Link
            json_file["link"]="https://fazarchiv.faz.net"+str(box_element.find_all("a",class_="globalLinkButton")[0].get("href"))
            #Kurztext
            json_file["text"]=box_element.find_all("p")[0].text[24:]
            #Datum
            json_file["date"]=box_element.find_all("li")[0].text[:-1]
            #Ressort
            json_file["ressort"]=box_element.find_all("li")[1].text
            #Länge
            try:
                json_file["laenge"]=box_element.find_all("li")[2].text[1:]
            except:
                print("extraction laenge failed")
            json_list.append(json_file)

//fazarchiv.faz.net/fazSearch/index/searchForm?q=suizid&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=2270#hitlist
Page: 229.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=suizid&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=2280#hitlist
Page: 230.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=suizid&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&offset=2290#hitlist
Page: 231.0
https://fazarchiv.faz.net/fazSearch/index/searchForm?q=suizid&search_in=&timePeriod=timeFilter&timeFilter=&DT_from=&DT_to=&KO%2CSO=&crxdefs=&NN=&CO%2C1E=&CN=&BC=&submitSearch=Suchen&maxHits=&sorting=&toggleFilter=&dosearch=new&of

In [144]:
df=pd.DataFrame(json_list)

In [145]:
df.head()

,title,thema_1,date,text,link,autor,art,ressort,laenge
0,Die Golden Gate Bridge summt zu laut,Anwohner beschweren sich,Juni 2020,"Die orange-rote Golden Gate Bridge, das Wahrze...",https://fazarchiv.faz.net/document/FAZN__20200...,N/A,Artikel,FAZ.NET,211 Wörter
1,Staatsanwalt beklagt fehlende Kooperation von ...,Im Fall Jeffrey Epstein,Juni 2020,Ein amerikanischer Ermittler hat die Darstellu...,https://fazarchiv.faz.net/document/FAZN__20200...,N/A,Artikel,FAZ.NET,330 Wörter
2,BIG FLOYD,NICHT JUGENDFREI,Juni 2020,VON FLORENTIN SCHUMACHER / / Bevor sein Tod i...,https://fazarchiv.faz.net#,N/A,Zeitungsseite,Sonntagszeitung,675 Wörter
3,Ode an die Verwilderung,,Juni 2020,Moralisch ist das eine Zwickmühle: Darf man es...,https://fazarchiv.faz.net#,Plötzlich empfindsam: Oskar Roehler schreibt m...,Zeitungsseite,Feuilleton,939 Wörter
4,Die Monster sind immer noch da draußen,,Mai 2020,"In der Dokumentation ""Jeffrey Epstein: Filthy ...",https://fazarchiv.faz.net#,N/A,Zeitungsseite,Feuilleton,809 Wörter


In [146]:
df["month"]=df["date"].apply(lambda x: get_month(x))
df["year"]=df["date"].apply(lambda x: x[-4:])
df["date"]=df.apply(lambda x: pd.to_datetime("01."+str(x["month"])+"."+str(x["year"]),format="%d.%m.%Y"),axis=1)
df.drop(["month","year"],axis=1,inplace=True)

In [147]:
df.head()

,title,thema_1,date,text,link,autor,art,ressort,laenge
0,Die Golden Gate Bridge summt zu laut,Anwohner beschweren sich,2020-06-01,"Die orange-rote Golden Gate Bridge, das Wahrze...",https://fazarchiv.faz.net/document/FAZN__20200...,N/A,Artikel,FAZ.NET,211 Wörter
1,Staatsanwalt beklagt fehlende Kooperation von ...,Im Fall Jeffrey Epstein,2020-06-01,Ein amerikanischer Ermittler hat die Darstellu...,https://fazarchiv.faz.net/document/FAZN__20200...,N/A,Artikel,FAZ.NET,330 Wörter
2,BIG FLOYD,NICHT JUGENDFREI,2020-06-01,VON FLORENTIN SCHUMACHER / / Bevor sein Tod i...,https://fazarchiv.faz.net#,N/A,Zeitungsseite,Sonntagszeitung,675 Wörter
3,Ode an die Verwilderung,,2020-06-01,Moralisch ist das eine Zwickmühle: Darf man es...,https://fazarchiv.faz.net#,Plötzlich empfindsam: Oskar Roehler schreibt m...,Zeitungsseite,Feuilleton,939 Wörter
4,Die Monster sind immer noch da draußen,,2020-05-01,"In der Dokumentation ""Jeffrey Epstein: Filthy ...",https://fazarchiv.faz.net#,N/A,Zeitungsseite,Feuilleton,809 Wörter


In [148]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\FAZ\FAZ_Archiv_Query_[Suizid].xlsx') as writer:
    df.to_excel(writer)